In [152]:
import pandas as pd
import gzip
import json
import ast
import re

In [153]:
archivo_comprimido = 'Json_comprimidos\steam_games.json.gz'
# Descomprimir y leer el primer archivo con gzip
with gzip.open(archivo_comprimido, 'rt', encoding='utf-8') as f:
    contenido_json = []
    for linea in f:
        contenido_json.append(json.loads(linea.strip()))

df_juegos = pd.DataFrame(contenido_json)
df_juegos.tail(3)

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
C:\Users\lucat\AppData\Local\Temp\ipykernel_7924\2920012719.py:1: SyntaxWarning: invalid escape sequence '\s'
  archivo_comprimido = 'Json_comprimidos\steam_games.json.gz'


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
120442,Laush Studio,"[Indie, Racing, Simulation]",Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,Laush Dmitriy Sergeevich
120443,SIXNAILS,"[Casual, Indie]",EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,"xropi,stev3ns"
120444,NaN,NaN,Maze Run VR,NaN,http://store.steampowered.com/app/681550/Maze_...,NaN,"[Early Access, Adventure, Indie, Action, Simul...",http://steamcommunity.com/app/681550/reviews/?...,"[Single-player, Stats, Steam Leaderboards, HTC...",4.99,True,681550,NaN


In [154]:
df_juegos.isnull().sum()

publisher       96362
genres          91593
app_name        88312
title           90360
url             88310
release_date    90377
tags            88473
reviews_url     88312
specs           88980
price           89687
early_access    88310
id              88312
developer       91609
dtype: int64

In [155]:
#Dropeamos las columnas que sabemos no nos van a servir. 
#los urls y el early acces no seran utilizados. 
#Publisher y title son redundantes, nos quedamos con app_name y developer que tienen menos datos nulos incluso. 
#Lo mismo ocurre con specs y genres con respecto a tag. Esta ultima columna repite las anteriores y tiene mas valores. 
df_juegos = df_juegos.drop(columns=['url', 'reviews_url', 'early_access','publisher','title','specs','tags'])
#Si no tenemos el nombre, id y developer no nos sirve porque no podemos recomendar un juego sin nombre.
columnas_obligatorias  = ['app_name','developer','id'] 
df_juegos = df_juegos.dropna(subset=columnas_obligatorias, how='all') #Tiene que tener los dos faltantes, app_name o titulo.

In [156]:
df_juegos.isnull().sum() 

genres          3282
app_name           1
release_date    2066
price           1377
id                 1
developer       3298
dtype: int64

In [157]:
#Dropeamos el id nulo que queda porque no vamos a poder hacer join 
df_juegos = df_juegos.dropna(subset=['id'])

In [158]:
df_juegos.isnull().sum() 

genres          3282
app_name           1
release_date    2066
price           1377
id                 0
developer       3298
dtype: int64

In [159]:
#Hacemos una funcion para extraer el año unicamente de la fecha. 

def extract_year(date_str):
    try:
        # Intentar convertir la cadena a una fecha
        date = pd.to_datetime(date_str, errors='raise')
        # Extraer el año y convertirlo a string
        date_str = str(date.year)
    except:
        # Si ocurre una excepción (porque no se puede convertir a fecha), retornar la cadena original
        pass
    if date_str == None:
        return None
    # Patrón para encontrar un año en cualquier lugar de la cadena
    year_pattern = r'\b\d{4}\b'
    
    # Intentar encontrar el año en cualquier parte de la cadena
    match = re.search(year_pattern, date_str)
    if match:
        return match.group()  # Devolver el año encontrado
    
    try:
        # Intentar convertir la cadena a una fecha
        date = pd.to_datetime(date_str, errors='raise')
        # Extraer el año y convertirlo a string
        return str(date.year)
    except:
        # Si ocurre una excepción (porque no se puede convertir a fecha), retornar None
        return None

# Aplicar la función a la columna release_date
df_juegos['release_date'] = df_juegos['release_date'].apply(extract_year)

C:\Users\lucat\AppData\Local\Temp\ipykernel_7924\198900554.py:6: UserWarning: Parsing dates in %d.%m.%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  date = pd.to_datetime(date_str, errors='raise')


In [160]:
#Extraemos los valores que no son numeros de la columna price. 

def es_string(valor):
    return isinstance(valor, str) and not valor.replace('.', '', 1).isdigit()

# Aplicar la función y filtrar los valores que son strings
lista_strings = df_juegos['price'].apply(lambda x: x if es_string(x) else None).dropna().tolist()

print(lista_strings)

df_juegos['price'] = df_juegos['price'].replace(to_replace=lista_strings, value='0.0') #Los reemplazamos por 0 y lo convertimos a float
df_juegos['price'] = df_juegos['price'].astype(float)
#Por ultimo nos queda desanidar la columna tag. 
df_juegos['genres'] = df_juegos['genres'].apply(lambda x: ', '.join(x) if isinstance(x, list) else '')
df_juegos.head(5)

['Free To Play', 'Free to Play', 'Free', 'Free', 'Free', 'Free', 'Free to Play', 'Free', 'Free', 'Free', 'Free to Play', 'Free to Play', 'Free to Play', 'Free to Play', 'Free', 'Free to Play', 'Free to Play', 'Free to Play', 'Free to Play', 'Free to Play', 'Free to Play', 'Free Demo', 'Free Demo', 'Play for Free!', 'Free to Play', 'Play for Free!', 'Free to Play', 'Free to Play', 'Free', 'Free', 'Free to Play', 'Free to Play', 'Free', 'Free to Play', 'Free to Play', 'Free to Play', 'Free to Play', 'Free To Play', 'Free', 'Free to Play', 'Free', 'Free To Play', 'Free', 'Free', 'Free to Play', 'Free to Play', 'Free to Play', 'Free to Play', 'Install Now', 'Free To Play', 'Free', 'Free to Play', 'Free', 'Free', 'Free To Play', 'Free to Play', 'Free to Play', 'Free', 'Free', 'Free to Play', 'Play WARMACHINE: Tactics Demo', 'Free', 'Free to Play', 'Free to Play', 'Free', 'Free', 'Free to Play', 'Free to Play', 'Free', 'Free', 'Free To Play', 'Free to Play', 'Free to Play', 'Free to Play', '

,genres,app_name,release_date,price,id,developer
88310,"Action, Casual, Indie, Simulation, Strategy",Lost Summoner Kitty,2018,4.99,761140,Kotoshiro
88311,"Free to Play, Indie, RPG, Strategy",Ironbound,2018,0.00,643980,Secret Level SRL
88312,"Casual, Free to Play, Indie, Simulation, Sports",Real Pool 3D - Poolians,2017,0.00,670290,Poolians.com
88313,"Action, Adventure, Casual",弹炸人2222,2017,0.99,767400,彼岸领域
88314,,Log Challenge,nan,2.99,773570,NaN


In [161]:
#Pasamos el primer dataset limpio y listo para usar de manera general, despues cada funcion tomará unicamente los datos relevantes. 
df_juegos.to_json('dataframe_juegos.json')

In [162]:
#Hacemos el primer Dataset para el primer endpoint con el JSON previamente depurado. Aun quedan cosas por depurar. 
df = pd.read_json('dataframe_juegos.json') #Leemos el archivo
df= df.dropna(subset=['release_date']) #Como la funcion pide contar articulos por año, si no tiene año no nos interesa.
df['release_date'] = df['release_date'].astype(int) #La pasamos a entero
df.drop(columns=['genres','id'], inplace=True) #Eliminamos consultas irrelevantes para la funcion

df.to_parquet('df_consulta_free.parquet') #Convertimos a parquet para que sea mas liviano

Empezamos con la limpieza del archivo numero 2

In [163]:
archivo = r'Json_comprimidos\users_items.json.gz'

data = []

with gzip.open(archivo, 'rt', encoding='utf-8') as file:
    for line in file:
        try:
            json_data = ast.literal_eval(line)
            data.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue 

df_items = pd.DataFrame(data)
df_items.head()

,user_id,items_count,steam_id,user_url,items
0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,76561198035864385,http://steamcommunity.com/id/js41637,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,76561198007712555,http://steamcommunity.com/id/evcentric,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,76561197963445855,http://steamcommunity.com/id/Riot-Punch,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,76561198002099482,http://steamcommunity.com/id/doctr,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [164]:
#Luego de analizar con las otras tablas observamos que el user_id es el que se usa, eliminando el steam_id que no será utilizado y el user_url 
df_items = df_items.drop(columns=['steam_id','user_url'])
#Chequeamos nulos 
df_items.isnull().sum()

user_id        0
items_count    0
items          0
dtype: int64

In [165]:
df_items.head(5)
#Hay que desanidar la columna items. 

,user_id,items_count,items
0,76561197970982479,277,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
1,js41637,888,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
2,evcentric,137,"[{'item_id': '1200', 'item_name': 'Red Orchest..."
3,Riot-Punch,328,"[{'item_id': '10', 'item_name': 'Counter-Strik..."
4,doctr,541,"[{'item_id': '300', 'item_name': 'Day of Defea..."


In [166]:
# Explode la columna 'items'
df_items_exploded = df_items.explode('items')
# Convertimos los valores a columnas
df_items_opiniones = df_items_exploded['items'].apply(pd.Series)
df_items_final = pd.DataFrame(df_items_opiniones)
df_items_final.head()

,item_id,item_name,playtime_forever,playtime_2weeks,0
0,10,Counter-Strike,6.0,0.0,NaN
0,20,Team Fortress Classic,0.0,0.0,NaN
0,30,Day of Defeat,7.0,0.0,NaN
0,40,Deathmatch Classic,0.0,0.0,NaN
0,50,Half-Life: Opposing Force,0.0,0.0,NaN


In [167]:
#Hacemos una copia porque es muy pesado el paso anterior, para no repetirlo 
df_items_final_copia = df_items_final.copy()

In [168]:
# Resetear el índice del DataFrame original para mantener el índice como columna
df_items_final_copia = df_items_final_copia.reset_index()
#Mappeamos los usuarios para poder hacer el merge luego de hacer el explode
mappear_items = df_items['user_id']
# Hacer el join basado en el índice del DataFrame original
df_items_final_copia = df_items_final_copia.join(mappear_items, on='index')

# Eliminamos el indice original ya no nos sirve
df_items_final_copia = df_items_final_copia.drop(columns=['index'])

df_items_final_copia.head()

,item_id,item_name,playtime_forever,playtime_2weeks,0,user_id
0,10,Counter-Strike,6.0,0.0,NaN,76561197970982479
1,20,Team Fortress Classic,0.0,0.0,NaN,76561197970982479
2,30,Day of Defeat,7.0,0.0,NaN,76561197970982479
3,40,Deathmatch Classic,0.0,0.0,NaN,76561197970982479
4,50,Half-Life: Opposing Force,0.0,0.0,NaN,76561197970982479


In [169]:
#Eliminamos las columnas que no nos van a interesar. 
df_items_final_copia = df_items_final_copia.drop(columns=[0,'playtime_2weeks'])


In [170]:
df_items_final_copia.to_json('dataframe_items_desanidados.json')

Realizamos la limpieza del ultimo DATASET

In [171]:
archivo = r'Json_comprimidos\user_reviews.json.gz'

data = []

with gzip.open(archivo, 'rt', encoding='utf-8') as file:
    for line in file:
        try:
            json_data = ast.literal_eval(line)
            data.append(json_data)
        except ValueError as e:
            print(f"Error en la línea: {line}")
            continue

df_reviews = pd.DataFrame(data)

In [172]:
df_reviews.head()

,user_id,user_url,reviews
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,"[{'funny': '', 'posted': 'Posted November 5, 2..."
1,js41637,http://steamcommunity.com/id/js41637,"[{'funny': '', 'posted': 'Posted June 24, 2014..."
2,evcentric,http://steamcommunity.com/id/evcentric,"[{'funny': '', 'posted': 'Posted February 3.',..."
3,doctr,http://steamcommunity.com/id/doctr,"[{'funny': '', 'posted': 'Posted October 14, 2..."
4,maplemage,http://steamcommunity.com/id/maplemage,"[{'funny': '3 people found this review funny',..."


In [173]:
#No nos interesa el user url
df_reviews = df_reviews.drop(columns=['user_url'])
#df_reviews.to_json('dataframe_reviews.json')

In [174]:
# Explode la columna 'reviews'
df_exploded = df_reviews.explode('reviews')

# Convierte los diccionarios en columnas
df_opiniones = df_exploded['reviews'].apply(pd.Series)

df_final = pd.DataFrame(df_opiniones)

df_final

,funny,posted,last_edited,item_id,helpful,recommend,review,0
0,,"Posted November 5, 2011.",,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,NaN
0,,"Posted July 15, 2011.",,22200,No ratings yet,True,It's unique and worth a playthrough.,NaN
0,,"Posted April 21, 2011.",,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...,NaN
1,,"Posted June 24, 2014.",,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...,NaN
1,,"Posted September 8, 2013.",,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...,NaN
...,...,...,...,...,...,...,...,...
25797,,Posted July 10.,,70,No ratings yet,True,a must have classic from steam definitely wort...,NaN
25797,,Posted July 8.,,362890,No ratings yet,True,this game is a perfect remake of the original ...,NaN
25798,1 person found this review funny,Posted July 3.,,273110,1 of 2 people (50%) found this review helpful,True,had so much fun plaing this and collecting res...,NaN
25798,,Posted July 20.,,730,No ratings yet,True,:D,NaN


In [175]:
#Dropeamos las columnas que no nos interesan. 
df_final = df_final.drop(columns=['last_edited','helpful','funny','last_edited',0])
df_final

,posted,item_id,recommend,review
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...
0,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.
0,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...
1,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...
1,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...
...,...,...,...,...
25797,Posted July 10.,70,True,a must have classic from steam definitely wort...
25797,Posted July 8.,362890,True,this game is a perfect remake of the original ...
25798,Posted July 3.,273110,True,had so much fun plaing this and collecting res...
25798,Posted July 20.,730,True,:D


In [176]:
#Hacemos una copia por las dudas. 
df_final_copia = df_final.copy()

In [177]:
mappear = df_reviews['user_id']
# Resetear el índice del DataFrame original para mantener el índice como columna
df_final_copia = df_final_copia.reset_index()

# Hacer el join basado en el índice del DataFrame original
df_final_copia = df_final_copia.join(mappear, on='index')

# Si no necesitas la columna de índice original, puedes eliminarla
df_final_copia = df_final_copia.drop(columns=['index'])

df_final_copia

,posted,item_id,recommend,review,user_id
0,"Posted November 5, 2011.",1250,True,Simple yet with great replayability. In my opi...,76561197970982479
1,"Posted July 15, 2011.",22200,True,It's unique and worth a playthrough.,76561197970982479
2,"Posted April 21, 2011.",43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,"Posted June 24, 2014.",251610,True,I know what you think when you see this title ...,js41637
4,"Posted September 8, 2013.",227300,True,For a simple (it's actually not all that simpl...,js41637
...,...,...,...,...,...
59328,Posted July 10.,70,True,a must have classic from steam definitely wort...,76561198312638244
59329,Posted July 8.,362890,True,this game is a perfect remake of the original ...,76561198312638244
59330,Posted July 3.,273110,True,had so much fun plaing this and collecting res...,LydiaMorley
59331,Posted July 20.,730,True,:D,LydiaMorley


In [178]:
df_final_copia.isna().sum()

posted       28
item_id      28
recommend    28
review       28
user_id       0
dtype: int64

In [179]:
#Dropeamos los na y pasamos a json para hacer las funciones posteriores
df_reviews_desanidadas = df_final_copia.dropna()
#Extraemos el año antes de guardarlo
df_reviews_desanidadas['posted'] = df_reviews_desanidadas['posted'].apply(extract_year)
df_reviews_desanidadas.head()

C:\Users\lucat\AppData\Local\Temp\ipykernel_7924\909527894.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews_desanidadas['posted'] = df_reviews_desanidadas['posted'].apply(extract_year)


,posted,item_id,recommend,review,user_id
0,2011,1250,True,Simple yet with great replayability. In my opi...,76561197970982479
1,2011,22200,True,It's unique and worth a playthrough.,76561197970982479
2,2011,43110,True,Great atmosphere. The gunplay can be a bit chu...,76561197970982479
3,2014,251610,True,I know what you think when you see this title ...,js41637
4,2013,227300,True,For a simple (it's actually not all that simpl...,js41637


In [180]:
df_reviews_desanidadas.to_json('dataframe_reviews_desanidadas.json')

Dataset para la función 2: Gasto de usuario, items y reviews presentadas 

In [181]:
df_items = pd.read_json('dataframe_items_desanidados.json')
df_reviews = pd.read_json('dataframe_reviews_desanidadas.json')
df_juegos = pd.read_json('dataframe_juegos.json')
#eliminamos las columnas que no necesitamos para la consulta dos
#solo queremos usuario, id, precio y si lo recomienda.
#df_items
df_items = df_items.drop(columns=['item_name','playtime_forever'])
#Si no tiene item_id no nos sirve. 
df_items = df_items.dropna(subset=['item_id']) 
df_items['item_id'] = df_items['item_id'].astype(int)
#df_reviews
#Eliminamos columnas que no nos interesa y los nulos de item_id para join 
df_reviews = df_reviews.drop(columns=['posted', 'review'])
df_reviews = df_reviews.dropna(subset=['item_id']) 
df_reviews['item_id']=df_reviews['item_id'].astype(int)
#df_juegos
#Repetimos el proceso y modificamos el nombre para el join
df_juegos = df_juegos.drop(columns=['genres','app_name', 'release_date','developer'])
df_juegos = df_juegos.dropna(subset=['id']) 
df_juegos['id'] = df_juegos['id'].astype(int)
df_juegos.rename(columns={'id':'item_id'}, inplace=True)

Hacemos los join

In [182]:
df_join = pd.merge(df_items, df_reviews, on=['user_id','item_id'], how='left')
#solo nos quedamos con los que tengan recomendacion True or False. Si no tiene recomendacion decidimos no asumir ninguno de los dos valores
#Esto hace la tabla mas pequeña y mas certera. 

In [183]:
df_final_consigna2 = pd.merge(df_join, df_juegos, on='item_id', how='inner')
#Si no tiene recomendacion le asignamos False
df_final_consigna2['recommend'] = df_final_consigna2['recommend'].fillna(False)

df_final_consigna2.head()

C:\Users\lucat\AppData\Local\Temp\ipykernel_7924\2651518945.py:3: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_final_consigna2['recommend'] = df_final_consigna2['recommend'].fillna(False)


,item_id,user_id,recommend,price
0,10,76561197970982479,False,9.99
1,20,76561197970982479,False,4.99
2,30,76561197970982479,False,4.99
3,40,76561197970982479,False,4.99
4,50,76561197970982479,False,4.99


In [184]:
#Creamos el segundo dataset 
df_final_consigna2.to_parquet('df_consulta_gasto_usuario.parquet')

Dataset para la Función N°3: maximo jugador para genero determinado y su cantidad por año

In [185]:
#necesitamos usuario, genero, año y horas jugadas
#hay que desanidar el género
df_juegos = pd.read_json('dataframe_juegos.json')
df_items = pd.read_json('dataframe_items_desanidados.json')

#Para este dataset no nos interesa el nombre.
df_items = df_items.drop(columns=['item_name'])

#Desanidamos genero, y nos quedamos con genero, año de lanzamiento y ID
df_juegos = df_juegos.drop(columns=['app_name', 'price','developer'])
df_juegos = df_juegos.dropna(subset=['id'])
df_juegos = df_juegos.dropna(subset=['release_date']) 
df_juegos['item_id'] = df_juegos['id'].astype(int)
df_juegos = df_juegos.drop(columns=['id'])
df_juegos['release_date'] = df_juegos['release_date'].astype(int)

In [186]:
#Hacemos el merge 
df_genero = pd.merge(df_items,df_juegos, on='item_id', how='inner')

df_genero = df_genero.drop(columns='item_id') #ya no nos interesa mas el item id

#Tambien podemos eliminar aquellas filas en el que playtime forever sea 0 porque no van a sumar nada

df_genero = df_genero[df_genero['playtime_forever']!= 0 ]

#Guardamos el 3 dataset en parquet 
df_genero.to_parquet('df_consulta_genero.parquet')


Dataset para la función N°4: Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)

En el dataset user_reviews se incluyen reseñas de juegos hechos por distintos usuarios. Debes crear la columna 'sentiment_analysis' aplicando análisis de sentimiento con NLP con la siguiente escala: debe tomar el valor '0' si es malo, '1' si es neutral y '2' si es positivo. Esta nueva columna debe reemplazar la de user_reviews.review para facilitar el trabajo de los modelos de machine learning y el análisis de datos. De no ser posible este análisis por estar ausente la reseña escrita, debe tomar el valor de 1.

In [187]:
df_reviews = pd.read_json('dataframe_reviews_desanidadas.json')
df_juegos = pd.read_json('dataframe_juegos.json')
#Podemos eliminar la columna helpful y las filas que tengan recommend False ya que no nos interesan los comentarios falses o neutros.
#Del DF de juegos no nos interesan ni el genero, ni el nombre ni el precio.
#Tenemos que transformar el año y el id a entero.
df_juegos = df_juegos.drop(columns=['genres','app_name','price'])
#Dropeamos los nan porque si falta el dato en cualquier de las columnas no nos sirve
df_juegos = df_juegos.dropna()
df_juegos['release_date'] = df_juegos['release_date'].astype(int)
df_juegos['release_date'] = df_juegos['release_date'].astype(str) #lo pasamos a string para poder hacer los joins.
df_juegos['item_id'] = df_juegos['id'].astype(int)
df_juegos = df_juegos.drop(columns=['id'])

In [188]:
 #si hacemos inner join nos elimina las filas que no tengan developer
df = pd.merge(df_reviews, df_juegos, on='item_id', how='inner')
#Hay 8000 filas que no tienen el año luego de realizada la funcion hay dos opciones: eliminarlas o asignarles el año de lanzamiento del juego. Elegimos la segunda opción.
df['posted'].fillna(df['release_date'], inplace=True)
df.head()
#Hacemos el join con el df de los juegos. 

C:\Users\lucat\AppData\Local\Temp\ipykernel_7924\3177080206.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['posted'].fillna(df['release_date'], inplace=True)
C:\Users\lucat\AppData\Local\Temp\ipykernel_7924\3177080206.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['2013' '2012' '2012' ... '2014' '2012' '2007']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df['posted'].fillna(df['release

,posted,item_id,recommend,review,user_id,release_date,developer
0,2011.0,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,2009,Tripwire Interactive
1,2011.0,22200,True,It's unique and worth a playthrough.,76561197970982479,2009,ACE Team
2,2013.0,227300,True,For a simple (it's actually not all that simpl...,js41637,2013,SCS Software
3,2013.0,239030,True,Very fun little game to play when your bored o...,js41637,2013,3909
4,2013,248820,True,A suitably punishing roguelike platformer. Wi...,evcentric,2013,"Hopoo Games, LLC"


In [189]:
#Hacemos una funcion que nos clasifique la review. 

import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
# Load the sentiment analyzer
sid = SentimentIntensityAnalyzer()

def classify_sentiment(review, recommend):
    if not recommend:
        return 0
    else:
        score = sid.polarity_scores(review)['compound']
        if score >= 0.05:
            return 2
        elif score > -0.05 and score < 0.05:
            return 1
        else:
            return 0
# Apply sentiment classification to the DataFrame
df['sentiment'] = df.apply(lambda row: classify_sentiment(row['review'], row['recommend']), axis=1)


In [190]:
#eliminamos la columna posted porque genera error y no nos servirá para el sistema de recomendacion 
df_copia = df.copy()
df_copia = df_copia.drop(columns=['posted'])

df_copia.to_parquet('df_consulta_sentimientos.parquet')

In [191]:
#nos quedamos con las columnas que nos interesan y con las filas que sean True y 2 de sentimiento
df = df.drop(columns=['review','release_date', 'item_id','user_id'])
df = df[df['recommend']== True]
df = df[df['sentiment']== 2]
#Al quedar solo los 2 y true podemos eliminar esas columnas. deberiamos contar developer por año y multiplicarlo por 2 para saber el puntaje. 
df = df.drop(columns=['recommend','sentiment'])
df = df.dropna(subset='posted')
df['year'] = df['posted'].astype(int)
df = df.drop(columns=['posted'])

df.to_parquet('df_consulta_positivo_desarrollador.parquet')

Función N° 5

def developer_reviews_analysis( desarrolladora : str ): Según el desarrollador, se devuelve un diccionario con el nombre del desarrollador como llave y una lista con la cantidad total de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento como valor positivo o negativo.

In [192]:
df_reviews = pd.read_json('dataframe_reviews_desanidadas.json')
df_juegos = pd.read_json('dataframe_juegos.json')
#Podemos eliminar la columna helpful y las filas que tengan recommend False ya que no nos interesan los comentarios falses o neutros.
# Debemos extraer la fecha del posted y el item_id para el join pero luego lo podemos eliminar. 
df_reviews = df_reviews.drop(columns=['posted'])
#Del DF de juegos no nos interesan ni el genero, ni el nombre ni el precio.
#Tenemos que transformar el año y el id a entero.
df_juegos = df_juegos.drop(columns=['genres','app_name','price','release_date'])
#Dropeamos los nan porque si falta el dato en cualquier de las columnas no nos sirve
df_juegos = df_juegos.dropna()
df_juegos['item_id'] = df_juegos['id'].astype(int)
df_juegos = df_juegos.drop(columns=['id'])
df_juegos.head()

,developer,item_id
88310,Kotoshiro,761140
88311,Secret Level SRL,643980
88312,Poolians.com,670290
88313,彼岸领域,767400
88315,Trickjump Games Ltd,772540


In [193]:
df = pd.merge(df_reviews, df_juegos, on='item_id', how='inner') 
#si hacemos inner join nos elimina las filas que no tengan developer
df['sentiment'] = df.apply(lambda row: classify_sentiment(row['review'], row['recommend']), axis=1)
df.head()


,item_id,recommend,review,user_id,developer,sentiment
0,1250,True,Simple yet with great replayability. In my opi...,76561197970982479,Tripwire Interactive,2
1,22200,True,It's unique and worth a playthrough.,76561197970982479,ACE Team,2
2,227300,True,For a simple (it's actually not all that simpl...,js41637,SCS Software,2
3,239030,True,Very fun little game to play when your bored o...,js41637,3909,2
4,248820,True,A suitably punishing roguelike platformer. Wi...,evcentric,"Hopoo Games, LLC",2


In [194]:
#Dropeamos las columnas que no nos sirven
df = df.drop(columns=['review','item_id','user_id','recommend'])
#No nos interesan los comentarios neutrales, solo los positivos y los negativos. 
df = df[df['sentiment']!=1]
df.to_parquet('df_consulta_sentimientos_desarrollador.parquet')